In [4]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import time

from requests import get

#Sometimes not including the header results in a failed response
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8'}

BASE_URL='https://www.goodreads.com'
LIST_URL='https://www.goodreads.com/list/show/1.Best_Books_Ever'
#LIST_URL='https://www.goodreads.com/list/best_of_year/2018?id=119307.Best_books_of_2018'

books={'URL':[]}

#the no. of pages this list has
num_pages=10

for i in range(1, num_pages):
    #time.sleep(120) #make sure you give enough time between page loads to not avoid server overload
    print(f'Reading page {i}')
    list_page_url=f'{LIST_URL}&page={i}'
    list_page=get(list_page_url, headers=hdr)
    list_soup = BeautifulSoup(list_page.content, 'html.parser')
    book_table=list_soup.find('table', attrs={'class':'tableList js-dataTooltip'})
    rows=book_table.find_all('tr')
    books['URL']+=[BASE_URL+r.find('a', attrs={'class':'bookTitle'}).attrs['href'] for r in rows]

books_df=pd.DataFrame.from_dict(books)
books_df.to_csv('books.csv')

Reading page 1
Reading page 2
Reading page 3
Reading page 4
Reading page 5
Reading page 6
Reading page 7
Reading page 8
Reading page 9


In [5]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import time
import os

from requests import get

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8'}

if not os.path.exists('book_data.csv'):
  book_data=pd.DataFrame(columns=[
    'image_url',
    'book_title',
    'book_authors',
    'book_rating',
    'book_rating_count',
    'book_review_count',
    'book_desc',
    'book_format',
    'book_edition',
    'book_pages',
    'book_isbn',
    'genres'
  ])

  book_data.to_csv('book_data.csv')

books=pd.read_csv('books.csv')
book_data=pd.read_csv('book_data.csv')

book_rows=[]
save_every=10

for i in range(len(book_data), len(books)):
    time.sleep(60)
    try:
        book_URL=books.at[i, 'URL']
        book_page=get(book_URL)
        book_soup=BeautifulSoup(book_page.content, 'html.parser')

        book=dict()

        #Save the URL of the image of the book cover to be downloaded later
        image_url=book_soup.find('img', attrs={'id':'coverImage'})
        if image_url:
            book['image_url']=image_url.attrs['src']
        else:
            book['image_url']=''

        #Title of the book
        book_title=book_soup.find('h1', attrs={'id':'bookTitle'})
        if book_title:
            book['book_title']=book_title.text.replace('\n','').strip()
        else:
            book['book_title']=''
        
        print(i, book['book_title'])

        #Author(s) of the book
        book['book_authors']='|'.join([a.find('span', attrs={'itemprop':'name'}).text for a in book_soup.find_all('a', attrs={'class':'authorName'})])
        
        #Rating given by users on goodreads
        book_rating=book_soup.find('span', attrs={'itemprop':'ratingValue'})
        if book_rating:
            book['book_rating']=book_rating.text.replace('\n','').strip()
        else:
            book['book_rating']=''
        book['book_rating_count']=book_soup.find('meta', attrs={'itemprop':'ratingCount'})['content']

        #No. of reviews for the book
        book['book_review_count']=book_soup.find('meta', attrs={'itemprop':'reviewCount'})['content']

        #A short description of the book, usually found on the back or inside cover of the book. Also called a blurb
        book_desc=book_soup.find('div', attrs={'class':'readable stacked'})
        if book_desc:
            book['book_desc']=book_desc.find_all('span')[-1].text
        else:
            book['book_desc']=''

        #Format of the book, e.g, paperback, hardcover, Kindle edition, etc.
        book_format=book_soup.find('div', attrs={'id':'details'}).find('span', attrs={'itemprop':'bookFormat'})
        if book_format:
            book['book_format']=book_format.text
        else:
            book['book_format']=''
        
        #Edition of the book
        book_edition=book_soup.find('div', attrs={'id':'details'}).find('span', attrs={'itemprop':'bookEdition'})
        if book_edition:
            book['book_edition']=book_edition.text
        else:
            book['book_edition']=''

        #No. of pages in the book
        book_pages=book_soup.find('div', attrs={'id':'details'}).find('span', attrs={'itemprop':'numberOfPages'})
        if book_pages:
            book['book_pages']=book_pages.text
        else:
            book['book_pages']=''
        
        #ISBN code of the book
        book_isbn=book_soup.find('div', attrs={'id':'bookDataBox'}).find('span', attrs={'itemprop':'isbn'})
        if book_isbn:
            book['book_isbn']=book_isbn.text
        else:
            book['book_isbn']=''

        #List of genres that the book belongs to. User supplied data.
        genres_list=book_soup.find_all('a', attrs={'class':'actionLinkLite bookPageGenreLink'})
        book['genres']='|'.join([i.text for i in genres_list])
        book_rows.append(book)

        if i%save_every==0:
            book_data.append(pd.DataFrame.from_dict(book_rows)).to_csv('book_data.csv', index=False)
            book_rows=[]
    except:
        book_data.append(pd.DataFrame.from_dict(book_rows)).to_csv('book_data.csv', index=False)
        book_rows=[]

10 The Fault in Our Stars
11 The Hitchhiker's Guide to the Galaxy
12 The Giving Tree
13 Wuthering Heights
14 The Da Vinci Code
15 Memoirs of a Geisha
16 The Picture of Dorian Gray
17 Alice's Adventures in Wonderland & Through the Looking-Glass
18 Jane Eyre
19 Les Misérables
20 Divergent
21 Fahrenheit 451
22 Romeo and Juliet
23 Lord of the Flies
24 The Alchemist
25 Crime and Punishment
26 The Perks of Being a Wallflower
27 The Great Gatsby
28 Ender's Game
29 City of Bones
30 The Help
31 Anne of Green Gables
32 Charlotte's Web
33 The Little Prince
34 The Time Traveler's Wife
35 Of Mice and Men
36 Harry Potter and the Sorcerer's Stone
37 Dracula
38 Brave New World
39 One Hundred Years of Solitude
40 The Princess Bride
41 The Catcher in the Rye
42 The Secret Garden
43 A Thousand Splendid Suns
44 The Lightning Thief
45 A Wrinkle in Time
46 A Game of Thrones
47 The Adventures of Huckleberry Finn
48 The Lovely Bones
49 The Outsiders
50 Where the Wild Things Are
51 Green Eggs and Ham
52 The Od

356 Lolita
357 Slaughterhouse-Five
358 Frankenstein: The 1818 Text
359 The Kite Runner
360 The Handmaid's Tale
361 The Giver
362 Catch-22
363 Dune
364 The Pillars of the Earth
365 The Stand
366 The Adventures of Sherlock Holmes
367 Watership Down
368 Great Expectations
369 One Flew Over the Cuckoo's Nest
370 The Bell Jar
371 Anna Karenina
372 Little Women
373 My Sister's Keeper
374 Outlander
375 The Girl with the Dragon Tattoo
376 Harry Potter and the Deathly Hallows
377 Matilda
378 The Fellowship of the Ring
379 Rebecca
380 The Color Purple
381 A Tree Grows in Brooklyn
382 1984
383 A Clockwork Orange
384 The Road
385 The Brothers Karamazov
386 Angela's Ashes
387 Vampire Academy
388 The Poisonwood Bible
389 Siddhartha
390 The Golden Compass
391 Don Quixote
392 Atlas Shrugged
393 The Old Man and the Sea
394 Harry Potter and the Prisoner of Azkaban
395 The Notebook
396 Winnie-the-Pooh
397 The Complete Stories and Poems
398 A Prayer for Owen Meany
399 Interview with the Vampire
400 The Hu

701 Harry Potter and the Order of the Phoenix
702 To Kill a Mockingbird
703 Pride and Prejudice
704 Twilight
705 The Book Thief
706 Animal Farm
707 The Chronicles of Narnia
708 J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings
709 Gone with the Wind
710 The Fault in Our Stars
711 The Hitchhiker's Guide to the Galaxy
712 The Giving Tree
713 Wuthering Heights
714 The Da Vinci Code
715 Memoirs of a Geisha
716 The Picture of Dorian Gray
717 Alice's Adventures in Wonderland & Through the Looking-Glass
718 Jane Eyre
719 Les Misérables
720 Divergent
721 Fahrenheit 451
722 Romeo and Juliet
723 Lord of the Flies
724 The Alchemist
725 Crime and Punishment
726 The Perks of Being a Wallflower
727 The Great Gatsby
728 Ender's Game
729 City of Bones
730 The Help
731 Anne of Green Gables
732 Charlotte's Web
733 The Little Prince
734 The Time Traveler's Wife
735 Of Mice and Men
736 Harry Potter and the Sorcerer's Stone
737 Dracula
738 Brave New World
739 One Hundred Years of Solitu

In [7]:

import pandas as pd
import urllib.request
import os

book_data=pd.read_csv('book_data.csv')
#PATH='C:\\Python\\Python37-32\\Scripts\\code\\images\\'
PATH='C:\\Users\\Yaolong-PC\\Downloads\\repository\\Data-Mining-Books\\images\\'
files=os.listdir(PATH)
n=0
if len(files)>0:
    n=max([int(f[:-4]) for f in os.listdir(PATH)])+1

for i in range(n, len(book_data)):
    url=book_data.at[i, 'image_url']
    filename=f'{i}.jpg'
    if not pd.isna(url):
        urllib.request.urlretrieve(url, PATH+filename)
    if i%100==0:
        print(i)

0
